<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [13]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error

In [14]:
HW8_PATH = '../../data/hw8_vw/'

In [66]:
!HW8_PATH=../../data/hw8_vw

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [19]:
!head -1 $HW8_PATH/train.json > $HW8_PATH/train1.json

In [20]:
with open('../../data/hw8_vw/train1.json') as inp_json:
    first_json = json.load(inp_json)

In [21]:
first_json.keys()

dict_keys(['author', 'meta_tags', 'polling', 'hubs', '_id', 'flags', 'published', 'domain', '_timestamp', 'tags', 'post_id', 'flow', 'content', 'link_tags', 'title', 'url'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [6]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [7]:
first_json['_timestamp']

1493192186.0903192

In [29]:
from datetime import datetime
dt = datetime.fromtimestamp(first_json['_timestamp'])
dt.month, dt.hour

(4, 7)

In [8]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [9]:
first_json['domain']

'habrahabr.ru'

In [27]:
first_json['published']['$date']

'2006-07-15T01:48:00.000Z'

In [11]:
first_json['title']

'Самопроизвольное разлогинивание'

In [12]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [13]:
first_json['polling']

In [14]:
first_json['post_id']

7

In [15]:
first_json['flags']

[]

In [16]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [17]:
first_json['flow']

In [18]:
first_json['tags']

['логин', 'login']

In [18]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [19]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [20]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [286]:
train_target = pd.read_csv(HW8_PATH + '/train_target.csv',
                          index_col='url')

In [23]:
train_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120000 entries, https://habrahabr.ru/post/7/ to https://geektimes.ru/post/87438/
Data columns (total 1 columns):
target    120000 non-null float64
dtypes: float64(1)
memory usage: 1.8+ MB


In [24]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину – постройте признак content_len – длина текста в миллионнах символов.
Также постройте признаки – час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [25]:
cyrillic_translit={
    u'\u0410': 'A', u'\u0430': 'a',
    u'\u0411': 'B', u'\u0431': 'b',
    u'\u0412': 'V', u'\u0432': 'v',
    u'\u0413': 'G', u'\u0433': 'g',
    u'\u0414': 'D', u'\u0434': 'd',
    u'\u0415': 'E', u'\u0435': 'e',
    u'\u0416': 'Zh', u'\u0436': 'zh',
    u'\u0417': 'Z', u'\u0437': 'z',
    u'\u0418': 'I', u'\u0438': 'i',
    u'\u0419': 'I', u'\u0439': 'i',
    u'\u041a': 'K', u'\u043a': 'k',
    u'\u041b': 'L', u'\u043b': 'l',
    u'\u041c': 'M', u'\u043c': 'm',
    u'\u041d': 'N', u'\u043d': 'n',
    u'\u041e': 'O', u'\u043e': 'o',
    u'\u041f': 'P', u'\u043f': 'p',
    u'\u0420': 'R', u'\u0440': 'r',
    u'\u0421': 'S', u'\u0441': 's',
    u'\u0422': 'T', u'\u0442': 't',
    u'\u0423': 'U', u'\u0443': 'u',
    u'\u0424': 'F', u'\u0444': 'f',
    u'\u0425': 'Kh', u'\u0445': 'kh',
    u'\u0426': 'Ts', u'\u0446': 'ts',
    u'\u0427': 'Ch', u'\u0447': 'ch',
    u'\u0428': 'Sh', u'\u0448': 'sh',
    u'\u0429': 'Shch', u'\u0449': 'shch',
    u'\u042a': '"', u'\u044a': '"',
    u'\u042b': 'Y', u'\u044b': 'y',
    u'\u042c': "'", u'\u044c': "'",
    u'\u042d': 'E', u'\u044d': 'e',
    u'\u042e': 'Iu', u'\u044e': 'iu',
    u'\u042f': 'Ia', u'\u044f': 'ia', 
    u'\u0401': 'Yo', u'\u0451': 'yo'
}

def transliterate(word, translit_table):
    converted_word = ''
    for char in word:
        transchar = ''
        if char in translit_table:
            transchar = translit_table[char]
        elif char == " ":
            transchar = char
        converted_word += transchar
    return converted_word

transliterate(u"Чёрная дыра»", cyrillic_translit)

'Chyornaia dyra'

In [289]:
def to_unicode(s):    
    return str(s.encode("raw_unicode_escape"))[2:-1].replace(":", "").replace("|", "").replace("\\\\", "\\")

def to_latin(s):
    s = s.replace(":", "").replace("|", "")
    return transliterate(s, cyrillic_translit)

def to_vw(data, label):
    dt = datetime.strptime(data["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")
    month = (dt.month - 1)/11
    weekday = dt.weekday()/6
    hour = dt.hour/23
    out = [
        label,
        "title {}".format(to_unicode(data["title"])),
        "tags {}".format(to_unicode(" ".join(data["tags"]))),
        "domain {}".format(data["domain"]),
        "flow {}".format(data["flow"] if data["flow"] else "None"),
        "author {}".format(data["author"]["nickname"]),
        "hubs {}".format(to_unicode(" ".join([d["title"] for d in data["hubs"]]))),
        "num content_len:{} month:{} dayofweek:{} hour:{}\n".format(
            round(len(data_json["content"])/1000000, 2), round(month, 4), round(weekday, 4), round(hour, 4))
    ]
    
    return " |".join(out)

assert to_latin(u"Чёрная дыра»") == "Chyornaia dyra"

In [284]:
# file names

OUT_TRAIN = 'scaled_habr_train.vw'
OUT_VALID = 'habr_valid.vw'
OUT_TEST = 'scaled_habr_test.vw'

!OUT_TRAIN='scaled_habr_train.vw'
!OUT_VALID='habr_valid.vw'
!OUT_TEST='scaled_habr_test.vw'

In [292]:
%%time
# create train data

from datetime import datetime
import random

n = 120000
# n = 3

with open(HW8_PATH + '/train.json') as inp_json, \
     open(HW8_PATH + '/' + OUT_TRAIN, 'w') as out_train:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь
        if n == 0: break
        n -= 1
         
        label = train_target.loc[data_json["url"]].to_string(index=False)
        out = to_vw(data_json, label)        
        out_train.write(out)


CPU times: user 2min 45s, sys: 12.8 s, total: 2min 58s
Wall time: 2min 57s


In [291]:
!head -3 $HW8_PATH/$OUT_TRAIN

0.693147 |title \u0421\u0430\u043c\u043e\u043f\u0440\u043e\u0438\u0437\u0432\u043e\u043b\u044c\u043d\u043e\u0435 \u0440\u0430\u0437\u043b\u043e\u0433\u0438\u043d\u0438\u0432\u0430\u043d\u0438\u0435 |tags \u043b\u043e\u0433\u0438\u043d login |domain habrahabr.ru |flow None |author @ptitov |hubs \u0425\u0430\u0431\u0440\u0430\u0445\u0430\u0431\u0440 |num content_len:0.0 month:0.5455 dayofweek:0.8333 hour:0.0435
1.098612 |title Stand-along c\u043e\u043e\u0431\u0449\u0435\u0441\u0442\u0432\u0430 \u043f\u0440\u043e\u0442\u0438\u0432 \u0441\u043e\u043e\u0431\u0449\u0435\u0441\u0442\u0432 \u0432 \u0440\u0430\u043c\u043a\u0430\u0445 \u0441\u043e\u0446\u0438\u0430\u043b\u044c\u043d\u044b\u0445 \u0441\u0435\u0442\u0435\u0439 |tags \u0441\u043e\u043e\u0431\u0449\u0435\u0441\u0442\u0432\u0430 \u0438\u043d\u0442\u0435\u0440\u043d\u0435\u0442-\u0441\u043e\u043e\u0431\u0449\u0435\u0441\u0442\u0432\u043e \u0441\u043e\u0446\u0438\u0430\u043b\u044c\u043d\u044b\u0435 \u0441\u0435\u0442\u0438 \u043d\u0438

In [256]:
# split train into train and validation

!split -l 100000 $HW8_PATH/$OUT_TRAIN $HW8_PATH/$OUT_TRAIN"_"

!mv $HW8_PATH/$OUT_TRAIN"_aa" $HW8_PATH/$OUT_TRAIN
!mv $HW8_PATH/$OUT_TRAIN"_ab" $HW8_PATH/$OUT_VALID

In [262]:
%%time
# from validation target file 

!cut -f 1 -d ' ' $HW8_PATH/$OUT_VALID > $HW8_PATH/habr_valid_target.txt

CPU times: user 0 ns, sys: 12 ms, total: 12 ms
Wall time: 211 ms


Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [295]:
# best model MAE=0.95048

!rm $HW8_PATH/habr.cache 
!vw -d $HW8_PATH/$OUT_TRAIN \
  --loss_function squared --ngram 2 --passes 3 -c --cache_file $HW8_PATH/habr.cache \
  -b 28 -l 0.43 --power_t 0.17 \
  -f $HW8_PATH/habr_model.vw

Generating 2-grams for all namespaces.
final_regressor = ../../data/hw8_vw//habr_model.vw
Num weight bits = 28
learning rate = 0.43
initial_t = 0
power_t = 0.17
decay_learning_rate = 1
creating cache_file = ../../data/hw8_vw//habr.cache
Reading datafile = ../../data/hw8_vw//scaled_habr_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.480453 0.480453            1            1.0   0.6931   0.0000       17
0.374371 0.268289            2            2.0   1.0986   0.5806       39
0.207948 0.041526            4            4.0   0.0000   0.1411       43
3.950226 7.692504            8            8.0   4.9698   0.5465       43
4.027346 4.104465           16           16.0   2.3026   1.8167       35
4.090295 4.153244           32           32.0   2.8904   4.0504       25
2.995788 1.901282           64           64.0   0.0000   1.8160       21
2.324753 1.653717          128 

In [294]:
!rm $HW8_PATH/habr.cache 
!vw -d $HW8_PATH/$OUT_TRAIN \
  --loss_function quantile --ngram 2 --passes 3 -c --cache_file $HW8_PATH/habr.cache \
  -b 28 -l 0.43 --power_t 0.17 \
  -f $HW8_PATH/habr_model.vw

Generating 2-grams for all namespaces.
final_regressor = ../../data/hw8_vw//habr_model.vw
Num weight bits = 28
learning rate = 0.43
initial_t = 0
power_t = 0.17
decay_learning_rate = 1
creating cache_file = ../../data/hw8_vw//habr.cache
Reading datafile = ../../data/hw8_vw//scaled_habr_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.346574 0.346574            1            1.0   0.6931   0.0000       17
0.318420 0.290266            2            2.0   1.0986   0.5181       39
0.209217 0.100015            4            4.0   0.0000   0.1333       43
0.681852 1.154486            8            8.0   4.9698   0.2675       43
0.921427 1.161003           16           16.0   2.3026   0.7092       35
1.235742 1.550056           32           32.0   2.8904   1.7273       25
1.086853 0.937964           64           64.0   0.0000   1.2757       21
0.880453 0.674054          128 

In [277]:
# predict for validation and MAE

!vw -i $HW8_PATH/habr_model.vw -t -d $HW8_PATH/$OUT_VALID \
-p $HW8_PATH/habr_valid_predictions.txt --quiet

from sklearn.metrics import mean_absolute_error

y_true = np.loadtxt(HW8_PATH + '/habr_valid_target.txt')
y_pred = np.loadtxt(HW8_PATH + '/habr_valid_predictions.txt')

mean_absolute_error(y_true, y_pred)

0.42947762150000007

In [296]:
%time
# create test data

n = 52913
with open(HW8_PATH + '/test.json') as inp_json, \
     open(HW8_PATH + '/' + OUT_TEST, 'w') as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь
        if n == 0: break     
        n -= 1
        
        out = to_vw(data_json, "1") 
        out_vw.write(out)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 179 µs


In [297]:
# predict for test

!vw -i $HW8_PATH/habr_model.vw -t -d $HW8_PATH/habr_test.vw \
-p $HW8_PATH/habr_test_predictions.txt --quiet

In [298]:
# create submission

sample_sub = pd.read_csv(HW8_PATH + '/sample_submission.csv', index_col='url')
your_submission = sample_sub.copy()
your_submission['target'] = np.loadtxt(HW8_PATH + '/habr_test_predictions.txt')
your_submission.to_csv(HW8_PATH + '/submit/scaled_squared_b_28_l_0.43_t_0.17_120000.csv')

In [271]:
# calc mean target

!cut -f 1 -d ' ' $HW8_PATH/original_habr_train.vw > $HW8_PATH/original_habr_train_target.txt
train_target = np.loadtxt(HW8_PATH + '/habr_test_predictions.txt')
train_target.mean()

2.2946041099540757